**Use GPU: Runtime -> Change runtime type -> GPU (Hardware Accelerator)**

Setup

In [1]:
!cat ~/.keras/keras.json

{
    "epsilon": 1e-07, 
    "floatx": "float32", 
    "image_data_format": "channels_last", 
    "backend": "tensorflow"
}

In [2]:
import keras
print(keras.__version__)

2.4.3


DCGAN

In [3]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2DTranspose
from keras.layers.convolutional import Conv2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Reshape

class DCGAN:
	@staticmethod
	def build_generator(dim, depth, channels=1, inputDim=100, outputDim=512):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (dim, dim, depth)
		chanDim = -1

		# first set of FC => RELU => BN layers
		model.add(Dense(input_dim=inputDim, units=outputDim))
		model.add(Activation("relu"))
		model.add(BatchNormalization())

		# second set of FC => RELU => BN layers, this time preparing
		# the number of FC nodes to be reshaped into a volume
		model.add(Dense(dim * dim * depth))
		model.add(Activation("relu"))
		model.add(BatchNormalization())

		# reshape the output of the previous layer set, upsample +
		# apply a transposed convolution, RELU, and BN
		model.add(Reshape(inputShape))
		model.add(Conv2DTranspose(32, (5, 5), strides=(2, 2), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))

		# apply another upsample and transposed convolution, but
		# this time output the TANH activation
		model.add(Conv2DTranspose(channels, (5, 5), strides=(2, 2), padding="same"))
		model.add(Activation("tanh"))

		# return the generator model
		return model

	@staticmethod
	def build_discriminator(width, height, depth, alpha=0.2):
		# initialize the model along with the input shape to be
		# "channels last"
		model = Sequential()
		inputShape = (height, width, depth)

		# first set of CONV => RELU layers
		model.add(Conv2D(32, (5, 5), padding="same", strides=(2, 2), input_shape=inputShape))
		model.add(LeakyReLU(alpha=alpha))

		# second set of CONV => RELU layers
		model.add(Conv2D(64, (5, 5), padding="same", strides=(2, 2)))
		model.add(LeakyReLU(alpha=alpha))

		# first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(512))
		model.add(LeakyReLU(alpha=alpha))

		# sigmoid layer outputting a single value
		model.add(Dense(1))
		model.add(Activation("sigmoid"))

		# return the discriminator model
		return model

Train DCGAN w/ MNIST

In [4]:
from keras.models import Model
from keras.layers import Input
from keras.optimizers import Adam
from keras.datasets import mnist
from sklearn.utils import shuffle
from imutils import build_montages
import numpy as np
import cv2
import os

In [15]:
def train_dcgan_mnist(output_folderpath, epochs=50, batch_size=128):
    # store the epochs and batch size in convenience variables
    NUM_EPOCHS = epochs
    BATCH_SIZE = batch_size

    # load the MNIST dataset and stack the training and testing data
    # points so we have additional training data
    print("[INFO] loading MNIST dataset...")
    ((trainX, _), (testX, _)) = mnist.load_data()
    trainImages = np.concatenate([trainX, testX])

    # add in an extra dimension for the channel and scale the images
    # into the range [-1, 1] (which is the range of the tanh
    # function)
    trainImages = np.expand_dims(trainImages, axis=-1)
    trainImages = (trainImages.astype("float") - 127.5) / 127.5

    # build the generator
    print("[INFO] building generator...")
    gen = DCGAN.build_generator(7, 64, channels=1)

    # build the discriminator
    print("[INFO] building discriminator...")
    disc = DCGAN.build_discriminator(28, 28, 1)
    discOpt = Adam(lr=0.0002, beta_1=0.5, decay=0.0002 / NUM_EPOCHS)
    disc.compile(loss="binary_crossentropy", optimizer=discOpt)

    # build the adversarial model by first setting the discriminator to
    # *not* be trainable, then combine the generator and discriminator
    # together
    print("[INFO] building GAN...")
    disc.trainable = False
    ganInput = Input(shape=(100,))
    ganOutput = disc(gen(ganInput))
    gan = Model(ganInput, ganOutput)

    # compile the GAN
    ganOpt = Adam(lr=0.0002, beta_1=0.5, decay=0.0002 / NUM_EPOCHS)
    gan.compile(loss="binary_crossentropy", optimizer=discOpt)

    # randomly generate some benchmark noise so we can consistently
    # visualize how the generative modeling is learning
    print("[INFO] starting training...")
    benchmarkNoise = np.random.uniform(-1, 1, size=(256, 100))

    # loop over the epochs
    for epoch in range(NUM_EPOCHS):
        # show epoch information and compute the number of batches per
        # epoch
        print("[INFO] starting epoch {} of {}...".format(epoch + 1, NUM_EPOCHS))
        batchesPerEpoch = int(trainImages.shape[0] / BATCH_SIZE)

        # loop over the batches
        for i in range(0, batchesPerEpoch):
            # initialize an (empty) output path
            p = None

            # select the next batch of images, then randomly generate
            # noise for the generator to predict on
            imageBatch = trainImages[i * BATCH_SIZE:(i + 1) * BATCH_SIZE]
            noise = np.random.uniform(-1, 1, size=(BATCH_SIZE, 100))

            # generate images using the noise + generator model
            genImages = gen.predict(noise, verbose=0)

            # concatenate the *actual* images and the *generated* images,
            # construct class labels for the discriminator, and shuffle
            # the data
            X = np.concatenate((imageBatch, genImages))
            y = ([1] * BATCH_SIZE) + ([0] * BATCH_SIZE)
            (X, y) = shuffle(X, y)

            # convert to numpy array as input for train_on_batch
            X = np.array(X)
            y = np.array(y)

            # train the discriminator on the data
            discLoss = disc.train_on_batch(X, y)

            # let's now train our generator via the adversarial model by
            # (1) generating random noise and (2) training the generator
            # with the discriminator weights frozen
            noise = np.random.uniform(-1, 1, (BATCH_SIZE, 100))
            ganLoss = gan.train_on_batch(noise, np.array([1] * BATCH_SIZE))

            # check to see if this is the end of an epoch, and if so,
            # initialize the output path
            if i == batchesPerEpoch - 1:
                p = [output_folderpath, "epoch_{}_output.png".format(str(epoch + 1).zfill(4))]

            # otherwise, check to see if we should visualize the current
            # batch for the epoch
            else:
                # create more visualizations early in the training
                # process
                if epoch < 10 and i % 25 == 0:
                    p = [output_folderpath, "epoch_{}_step_{}.png".format(str(epoch + 1).zfill(4), str(i).zfill(5))]

                # visualizations later in the training process are less
                # interesting
                elif epoch >= 10 and i % 100 == 0:
                    p = [output_folderpath, "epoch_{}_step_{}.png".format(str(epoch + 1).zfill(4), str(i).zfill(5))]

            # check to see if we should visualize the output of the
            # generator model on our benchmark data
            if p is not None:
                # show loss information
                print("[INFO] Step {}_{}: discriminator_loss={:.6f}, adversarial_loss={:.6f}".format(epoch + 1, i, discLoss, ganLoss))

                # make predictions on the benchmark noise, scale it back
                # to the range [0, 255], and generate the montage
                images = gen.predict(benchmarkNoise)
                images = ((images * 127.5) + 127.5).astype("uint8")
                images = np.repeat(images, 3, axis=-1)
                vis = build_montages(images, (28, 28), (16, 16))[0]

                # write the visualization to disk
                p = os.path.sep.join(p)
                cv2.imwrite(p, vis)

    print("[INFO] done.")

In [16]:
train_dcgan_mnist(output_folderpath="drive/MyDrive/pyimagesearch/output/34-dcgan-mnist")

[INFO] loading MNIST dataset...
[INFO] building generator...
[INFO] building discriminator...
[INFO] building GAN...
[INFO] starting training...
[INFO] starting epoch 1 of 50...
[INFO] Step 1_0: discriminator_loss=0.723051, adversarial_loss=0.668614
[INFO] Step 1_25: discriminator_loss=0.029240, adversarial_loss=2.033889
[INFO] Step 1_50: discriminator_loss=0.001854, adversarial_loss=0.918732
[INFO] Step 1_75: discriminator_loss=2.205439, adversarial_loss=0.000179
[INFO] Step 1_100: discriminator_loss=0.042607, adversarial_loss=0.413303
[INFO] Step 1_125: discriminator_loss=0.026780, adversarial_loss=0.593967
[INFO] Step 1_150: discriminator_loss=0.033350, adversarial_loss=1.414719
[INFO] Step 1_175: discriminator_loss=0.035503, adversarial_loss=3.116061
[INFO] Step 1_200: discriminator_loss=0.456335, adversarial_loss=4.507927
[INFO] Step 1_225: discriminator_loss=0.388223, adversarial_loss=3.880646
[INFO] Step 1_250: discriminator_loss=0.377872, adversarial_loss=2.381344
[INFO] Step 1